# DAC Contest
This reference design will help you walk through a design flow of DAC SDC 2023. This is a simplified design to help users get started on the FPGA platform and to understand the overall flow. It does not contain any object detection hardware.

If you have any questions, please post on the Slack page (link on SDC website sidebar).

### Hardware

### Software
Note:
  * You will not submit your `dac_sdc.py` file, so any changes you make to this file will not be considered during evluation.  
  * You can use both PS and PL side to do inference.

### Object Detection

Object detection will be done on images in batches:
  * You will provide a Python callback function that will perform object detection on batch of images.  This callback function wile be called many times.
  * The callback function should return the locations of all images in the batch.
  * Runtime will be recorded during your callback function.
  * Images will be loaded from SD card before each batch is run, and this does not count toward your energy usage or runtime.
  
### Notebook
Your notebook should contain 4 code cells:

1. Importing all libraries and creating your Team object.
1. Downloading the overlay, compile the code, and performany any one-time configuration.
1. Python callback function and any other Python helper functions.
1. Running object detection
1. Cleanup



## 1. Imports and Create Team

In [1]:
import sys
import os

sys.path.append(os.path.abspath("../common"))

import time
from pathlib import Path
from functools import partial
import numpy as np

import dac_sdc
from iris_utils_v6 import TensorRTEngine

team_name = 'SKKU_IRIS_GPU'
dac_sdc.BATCH_SIZE = 1
team = dac_sdc.Team(team_name)

**Your team directory where you can access your notebook, and any other files you submit, is available as `team.team_dir`.**

## 2. Preparing the library and model
Prepare the dependencies for the contest, including installing python packages, compiling your binaries, and linking to the notebook.

Your team is responsible to make sure the correct packages are installed. For the contest environment, use the configuration below provided by Nvidia:
- [JetPack 4.6.1](https://developer.nvidia.com/embedded/jetpack-sdk-461)
    - Ubuntu 18.04
    - CUDA 10.2
    - cuDNN 8.2.1
    - gcc 7.5.0
    - python 3.6.9
    - TensorRT 8.2.1

## 3. Python Callback Function and Helper Functions


### Pushing the picture through the pipeline
In this example, we use contiguous memory arrays for sending and receiving data via DMA.

The size of the buffer depends on the size of the input or output data.  The example images are 640x360 (same size as training and test data), and we will use `pynq.allocate` to allocate contiguous memory.

### Callback function
The callback function:
  - Will be called on each batch of images (will be called many times)
  - Is prvided with a list of tuples of (image path, RGB image)
  - It should return a dictionary with an entry for each image:
    - Key: Image name (`img_path.name`)
    - Value: Dictionary of item type and bounding box (keys: `type`, `x`, `y`, `width`, `height`)

See the code below for an example:


In [2]:
def create_callback(engine_path):
    engine = TensorRTEngine(engine_path)
    input_shape = engine.input_shape # (1, 3, h, w)

    def callback(inputs):
        outputs = dict()
        for img_path, rgb_img in inputs:
            orig_img_shape = engine.preprocess(rgb_img)
            preds = engine.run()
            outputs[img_path.name] = engine.postprocess(preds, orig_img_shape)
        return outputs
    
    return callback, input_shape

In [3]:
engine_path = 'IRIS.trt'

iris_callback, input_shape = create_callback(engine_path)
_, c, h, w = input_shape
print("warmup")
iris_callback(
    [(Path(''), np.random.randint(low=0, high=255, size=(h, w, c), dtype=np.uint8)) for _ in range(300)]
)
print("warmup done")

warmup
warmup done


## 4. Running Object Detection

Call the following function to run the object detection.  Extra debug output is enabled when `debug` is `True`.

In [ ]:
team.run(iris_callback, debug=False)